Example solved by the Bramble-Pasciak Transformation

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

from netgen.geom2d import SplineGeometry
from netgen.occ import *

dim = 3

if dim==2:
    geo = SplineGeometry()
    geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
    geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
    mesh = Mesh( geo.GenerateMesh(maxh=0.02))
else:
    box = Box((0,0,0), (2,0.41,0.41))
    box.faces.name="wall"
    box.faces.Min(X).name="inlet"
    box.faces.Max(X).name="outlet"
    cyl = Cylinder((0.2,0,0.2), Y, h=0.41,r=0.05)
    cyl.faces.name="cyl"
    shape = box-cyl
    DrawGeo (shape)
    
    mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.02))
    Draw(mesh)

In [ ]:
V = VectorH1(mesh, order=1, dirichlet="wall|inlet|cyl")
V1 = H1(mesh, order=1, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=1)

u,v = V.TnT()
u1,v1 = V1.TnT()
p,q = Q.TnT()

h = specialcf.mesh_size
# bfa = BilinearForm(InnerProduct(grad(u),grad(v))*dx).Assemble()
bfa1 = BilinearForm(InnerProduct(grad(u1),grad(v1))*dx).Assemble()
bfb = BilinearForm(div(u)*q*dx).Assemble()
bfc = BilinearForm(h*h*grad(p)*grad(q)*dx).Assemble()
# prea = Preconditioner(bfa, "local")
# prea = Preconditioner(bfa, "direct", inverse="sparsecholesky")
# bfa.Assemble()

prea1 = Preconditioner(bfa1, "direct", inverse="sparsecholesky")
bfa1.Assemble()
mata = 0
prea = 0

for i in range(mesh.dim):
    E = Embedding(mesh.dim*V1.ndof, IntRange(i*V1.ndof, (i+1)*V1.ndof))
    mata = mata + E@bfa1.mat@E.T
    prea = prea + E@prea1@E.T

bfschur = BilinearForm(p*q*dx, diagonal=True).Assemble()
preschur = bfschur.mat.Inverse()

In [ ]:
from ngsolve.krylovspace import BramblePasciakCG

gfu = GridFunction(V)
gfp = GridFunction(Q)

if mesh.dim==2:
    uin = (1.5*4*y*(0.41-y)/(0.41*0.41), 0)
else:
    uin = (1.5*4*y*(0.41-y)/(0.41*0.41)*z*(0.41-z)/0.41**2,0, 0)

gfu.Set(uin, definedon=mesh.Boundaries("inlet"))

resf = (-mata * gfu.vec).Evaluate()
resg = (-bfb.mat * gfu.vec).Evaluate()

with TaskManager(pajetrace=10**8):
    sol = BramblePasciakCG (A=mata, B=bfb.mat, C=bfc.mat, f=resf, g=resg, \
                            preA=prea, preS=preschur, printrates='\r')

gfu.vec.data += sol[0]
gfp.vec.data += sol[1]

In [ ]:
Draw (Norm(gfu), mesh, order=1, clipping={"y":1,"z":0})
Draw (gfp, order=1);